#### 데이터 기본 정보

In [19]:
import pandas as pd

train_file_path = '/Users/imdaegyeong/Desktop/archive/Train.csv'
test_file_path = '/Users/imdaegyeong/Desktop/archive/Test.csv'

train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

train_data_info = train_data.head()
test_data_info = test_data.head()

train_data_info, test_data_info


(   feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
 0        100        160     1.6000          0          0          0   
 1         20         83     4.1500          1          0          0   
 2         99        150     1.5151          1          0          0   
 3         40         40     1.0000          0          0          0   
 4         12        234    19.5000          1          0          0   
 
    feature_7  feature_8  feature_9  feature_10  ...  feature_1550  \
 0          0          0          0           0  ...             0   
 1          0          0          0           1  ...             0   
 2          0          0          0           0  ...             0   
 3          0          0          0           0  ...             0   
 4          0          0          0           0  ...             0   
 
    feature_1551  feature_1552  feature_1553  feature_1554  feature_1555  \
 0             0             0             0             0          

#### 결측치 확인

In [20]:
# 결측값 확인
train_missing_values = train_data.isnull().sum()
test_missing_values = test_data.isnull().sum()

# 결측값이 존재하는 열과 그 개수만 확인
train_missing = train_missing_values[train_missing_values > 0]
test_missing = test_missing_values[test_missing_values > 0]

train_missing, test_missing


(Series([], dtype: int64), Series([], dtype: int64))

#### 피어슨 상관계수로 상관관계 높은 특성 도출, 먼저 단순하게 테스트 하기 위해 상위 10개로만 테스트 진행
#### PCA 방법 도입 > 10가지로 국한하지 말것

In [21]:
# 피어슨 상관계수를 이용하여 상관관계 분석
correlation_matrix = train_data.corr()

# Class와의 상관관계가 높은 상위 10개의 특성을 추출
top_features = correlation_matrix["Class"].abs().sort_values(ascending=False).head(11)  # Class 포함
top_features


Class           1.000000
feature_1400    0.424960
feature_1244    0.397658
feature_1154    0.368261
feature_1048    0.364530
feature_1144    0.364444
feature_1345    0.357316
feature_1423    0.356812
feature_1425    0.356812
feature_1199    0.356812
feature_1155    0.356812
Name: Class, dtype: float64

#### 이상치를 IQR 기법으로 처리하려 하였으나, 데이터가 0과 1로 이루어진 이진 데이터 이므로 다른 방법을 찾아야 함

#### 모델 기반 이상치 탐지 : 데이터 학습 과정에서 이상치를 감지하고, 모델이 그 영향을 최소화 하도록 하는 방식. 이상치를 명시적으로 하지 않고도 모델이 역할 수행. 이는 이진 데이터에서 적합하다. 데이터가 이진 또는 다차원일때 추천. 이 경우 랜덤 포레스트나 XGBoost 모델 자체로 이상치 처리하기.

##### 랜덤포레스트 / 상위 10개 특성

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 랜덤 포레스트 모델 생성
rf_model = RandomForestClassifier(random_state=42)

# 상위 10개의 선택된 특성으로 모델 학습
X_rf = train_data[selected_features]
y_rf = train_data['Class']

# 교차 검증을 사용하여 모델 성능 평가 (5-fold 교차 검증)
cross_val_scores = cross_val_score(rf_model, X_rf, y_rf, cv=5)

# 교차 검증 평균 성능 확인
cross_val_scores.mean()


0.9228544295647694

In [40]:
# 피어슨 상관계수를 이용하여 상관관계 분석
correlation_matrix = train_data.corr()

# Class와의 상관관계가 높은 상위 10개의 특성을 추출
top_features = correlation_matrix["Class"].abs().sort_values(ascending=False).head(11)  # Class 포함

# 상위 10개의 특성만 선택
selected_features = ['feature_1400', 'feature_1244', 'feature_1154', 'feature_1048', 'feature_1144', 
                     'feature_1345', 'feature_1423', 'feature_1425', 'feature_1199', 'feature_1155']

# 랜덤 포레스트 모델 생성
rf_model = RandomForestClassifier(random_state=42)

# 상위 10개의 선택된 특성으로 모델 학습
X_rf = train_data[selected_features]
y_rf = train_data['Class']

# 교차 검증을 사용하여 모델 성능 평가 (5-fold 교차 검증)
cross_val_scores = cross_val_score(rf_model, X_rf, y_rf, cv=5)

# 교차 검증 평균 성능 확인
cross_val_scores.mean()


0.9228544295647694

###### 교차검증 : 5-fold 교차검증 : 훈련데이터를 5개부분으로 나누고, 매번 다른 하나의 부분을 검증 데이터로 사용하여 성능을 평가한 후 평균을 내는 방법. 80:20 이렇게 하면 모델이 훈련 데이터에 오버피팅 되지않고 신뢰성 있는 성능 평가가 가능. 성능 확인을 위해 교차검증을 진행한것. 최종 성능은 테스트 데이터를 사용. 모델이 훈련데이터에서 얼마나 안정적인 성능을 낼수 있는지 확인하는 과정이 교차검증. 정밀도, 재현율, f1-score등 다른 성능 지표도 함께 평가해야 한다.

##### 이상치 탐지 성능을 높이기 위해 랜덤포레스트에서 선택 가능한 방법
###### 1. 하이퍼파라미터 튜닝
###### 2. 이상치 탐지에 집중한 임계값 조정
###### 3. 앙상블 기법 활용 (내부적으로 이용)
###### 4. 데이터 불균형 처리
###### 5. 특성 중요도 분석 및 추가 특성 선택

##### 1. 하이퍼 파라미터 튜닝
###### GridSearchCV : 완전탐색으로 최적의 파라미터 찾기/ 시간이 오래 걸릴수 있다.
###### RandomizedSearchCV : 설정된 범위 내에서 임의의 파라미터 조합을 일부만 탐색하여 최적의 파라미터를 빠르게 찾을 수 있다. 최적의 파라미터를 놓칠수도.

In [23]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델 설정
rf_model = RandomForestClassifier(random_state=42)

# 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# 1. GridSearchCV 설정
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)

# 2. RandomizedSearchCV 설정 (랜덤 탐색에서 반복 횟수 10 설정)
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=10, cv=5, scoring='f1', n_jobs=-1, random_state=42)

# 실행
grid_search.fit(X_rf, y_rf)
random_search.fit(X_rf, y_rf)

# 최적의 파라미터 확인
best_params_grid = grid_search.best_params_
best_params_random = random_search.best_params_

print("GridSearchCV Best Params:", best_params_grid)
print("RandomizedSearchCV Best Params:", best_params_random)


GridSearchCV Best Params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
RandomizedSearchCV Best Params: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 20}


##### GridSearchCV로 나온 최적의 파라미터로 모델 학습 진행

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# 최적의 파라미터로 랜덤 포레스트 모델 생성
rf_best_grid = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=4,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

# 모델 학습
rf_best_grid.fit(X_rf, y_rf)

# 교차 검증을 사용하여 F1-Score 계산 (5-fold 교차 검증)
f1_scores = cross_val_score(rf_best_grid, X_rf, y_rf, cv=5, scoring='f1')

# F1-Score 평균 계산
f1_score_mean = f1_scores.mean()
print("Average F1-Score:", f1_score_mean)


Average F1-Score: 0.41536186575614426


##### 최적의 임계값을 찾기. 현재 f1-score 값의 결과로 보았을때 이상치 탐지 성능이 아직 최적화 되지 않았을 가능성이 높음
###### 정밀도 : 양성(이상치)라고 예측한것 중에 실제 양성의 비율
###### 재현율 : 양성(이상치) 데이터 중에서 모델이 정확히 탐지한 비율
###### 지금 데이터에서 이상치가 적고 정상데이터가 많기 때문에 이상치 탐지를 잘 못하고 있을 가능성이 높다. 따라서 재현율이 낮을 가능성이 높은것.

In [25]:
from sklearn.metrics import precision_recall_curve
import numpy as np

# 예측 확률값을 계산하여 임계값 조정을 위한 준비
y_rf_probs = rf_best_grid.predict_proba(X_rf)[:, 1]  # 양성 클래스(1)의 확률값만 추출

# Precision-Recall Curve를 계산하여 다양한 임계값에서의 정밀도, 재현율 확인
precision, recall, thresholds = precision_recall_curve(y_rf, y_rf_probs)

# F1-Score를 기반으로 최적의 임계값을 선택
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]

print("Best Threshold:", best_threshold)
print("Max F1-Score:", max(f1_scores))


Best Threshold: 0.1799613075205644
Max F1-Score: 0.49498327759197325


##### f1-score 이 0.494 정도로 개선 되었지만 현재 데이터에서 이상치가 적고 정상데이터가 많으므로 이상치 탐지를 놓치는 경우가 발생. 이 경우 재현율이 낮아져 f1-score가 낮게 나오게 됨.따라서 모델 복잡성을 높이는 등의 방법을 사용하는 것이 좋다.

##### 지금 상황에서 이상치 데이터가 적으므로 SMOTE와 같은 오버샘플링 기법을 이용하여 모델 이상치를 더 잘 학습 시키거나, 언더샘플링을 통해 정상 데이터를 줄여서 데이터 균형을 맞추고 이상치 탐지 성능을 향상 시키는 방법이 있음 그게 아니라면 XGBoost와 같은 더 강력한 모델을 사용하는 것이 좋다.

##### SMOTE를 이용하여 오버샘플링 진행

###### SMOTE 원리: 1. K-NN에 기반하여 각 소수 클래스 샘플의 k개의 가까운 이웃을 찾는. 2. 새로운 샘플을 생성  : 원래의 소수 클래스 샘플과 그 이웃 샘플 사이의 직선상에 새로운 샘플을 생성, 즉 기존 데이터 포인트들 사이에 새로운 데이터를 삽입하는 방식. 3. 균형 맞추기 : 소수 데이터를 점진적으로 늘려 데이터의 불균형을 해결

In [26]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_rf, y_rf)

# 랜덤 포레스트 모델 학습 (GridSearchCV에서 찾은 최적의 파라미터 사용)
rf_model_smote = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=4,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

# SMOTE가 적용된 데이터로 모델 학습
rf_model_smote.fit(X_resampled, y_resampled)

# 교차 검증을 사용하여 F1-Score 계산 (5-fold 교차 검증)
f1_scores_smote = cross_val_score(rf_model_smote, X_resampled, y_resampled, cv=5, scoring='f1')

# F1-Score 평균 확인
f1_score_smote_mean = f1_scores_smote.mean()
print("F1-Score with SMOTE:", f1_score_smote_mean)


F1-Score with SMOTE: 0.6391543416329368


##### 그리드 서치로 다시 하이퍼파라미터 튜닝. f1-score를 높이기 위함

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델 설정 (SMOTE 적용 후)
rf_model_smote = RandomForestClassifier(random_state=42)

# 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# GridSearchCV 설정
grid_search_smote = GridSearchCV(estimator=rf_model_smote, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)

# GridSearchCV 실행
grid_search_smote.fit(X_resampled, y_resampled)

# 최적의 파라미터 확인
best_params_grid_smote = grid_search_smote.best_params_
print("Best Params with SMOTE:", best_params_grid_smote)


Best Params with SMOTE: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}


In [28]:
from sklearn.metrics import precision_recall_curve
import numpy as np

# 새로 찾은 최적의 하이퍼파라미터로 랜덤 포레스트 모델 학습
rf_best_smote = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=4,
    min_samples_split=2,
    n_estimators=300,
    random_state=42
)

# SMOTE가 적용된 데이터로 모델 학습
rf_best_smote.fit(X_resampled, y_resampled)

# 예측 확률값을 계산하여 임계값 조정 준비
y_rf_probs_smote = rf_best_smote.predict_proba(X_resampled)[:, 1]  # 양성 클래스(1)의 확률값만 추출

# Precision-Recall Curve를 계산하여 다양한 임계값에서의 정밀도, 재현율 확인
precision_smote, recall_smote, thresholds_smote = precision_recall_curve(y_resampled, y_rf_probs_smote)

# F1-Score를 기반으로 최적의 임계값을 선택
f1_scores_smote = 2 * (precision_smote * recall_smote) / (precision_smote + recall_smote)
best_threshold_smote = thresholds_smote[np.argmax(f1_scores_smote)]

# 최적의 임계값과 최대 F1-Score 확인
print("Best Threshold with SMOTE:", best_threshold_smote)
print("Max F1-Score with SMOTE:", max(f1_scores_smote))

# 정밀도와 재현율 출력
best_precision = precision_smote[np.argmax(f1_scores_smote)]
best_recall = recall_smote[np.argmax(f1_scores_smote)]
print("Best Precision:", best_precision)
print("Best Recall:", best_recall)


Best Threshold with SMOTE: 0.3572539068623872
Max F1-Score with SMOTE: 0.6666666666666666
Best Precision: 0.5
Best Recall: 1.0


##### 비정상적인 재현율로 다시 임계값 조정

In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 임계값을 더 높게 설정하여 재조정
higher_threshold = best_threshold_smote + 0.1  # 기존 임계값보다 0.1 높여서 시도

# 새로운 임계값으로 예측 수행 (확률이 higher_threshold 이상인 경우만 양성으로 예측)
y_pred_higher_threshold = (y_rf_probs_smote >= higher_threshold).astype(int)

# 정밀도, 재현율, F1-Score 계산
precision_higher = precision_score(y_resampled, y_pred_higher_threshold)
recall_higher = recall_score(y_resampled, y_pred_higher_threshold)
f1_higher = f1_score(y_resampled, y_pred_higher_threshold)

# 성능 출력
print("Precision with higher threshold:", precision_higher)
print("Recall with higher threshold:", recall_higher)
print("F1-Score with higher threshold:", f1_higher)


Precision with higher threshold: 0.9089874857792947
Recall with higher threshold: 0.49320987654320986
F1-Score with higher threshold: 0.6394557823129251


##### 언더샘플링을 해보고 성능 비교

In [30]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve
import numpy as np

# 언더샘플링 적용
undersampler = RandomUnderSampler(random_state=42)
X_resampled_under, y_resampled_under = undersampler.fit_resample(X_rf, y_rf)

# 언더샘플링된 데이터로 랜덤 포레스트 모델 학습 (최적의 하이퍼파라미터 적용)
rf_model_under = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=4,
    min_samples_split=2,
    n_estimators=300,
    random_state=42
)

rf_model_under.fit(X_resampled_under, y_resampled_under)

# 예측 확률값 계산
y_rf_probs_under = rf_model_under.predict_proba(X_resampled_under)[:, 1]  # 양성 클래스(1)의 확률값만 추출

# Precision-Recall Curve를 계산하여 임계값 조정
precision_under, recall_under, thresholds_under = precision_recall_curve(y_resampled_under, y_rf_probs_under)

# F1-Score를 기반으로 최적의 임계값 선택
f1_scores_under = 2 * (precision_under * recall_under) / (precision_under + recall_under)
best_threshold_under = thresholds_under[np.argmax(f1_scores_under)]

# 최적의 임계값에서 예측 수행
y_pred_under = (y_rf_probs_under >= best_threshold_under).astype(int)

# 성능 평가 (정밀도, 재현율, F1-Score)
precision_under_final = precision_score(y_resampled_under, y_pred_under)
recall_under_final = recall_score(y_resampled_under, y_pred_under)
f1_under_final = f1_score(y_resampled_under, y_pred_under)

# 결과 출력
print("Best Precision (UnderSampling):", precision_under_final)
print("Best Recall (UnderSampling):", recall_under_final)
print("Best F1-Score (UnderSampling):", f1_under_final)


Best Precision (UnderSampling): 0.5
Best Recall (UnderSampling): 1.0
Best F1-Score (UnderSampling): 0.6666666666666666


##### 랜덤 포레스트 결과 : 하이퍼파라미터와 임계치를 조정을 하였음에도 F1-score 및 정밀도 재현율 부분에서 높은 값을 나타내지 못하는것 같아 다른 모델을 사용하는것이 좋을것으로 예상됨